In [45]:
# Dependencies
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

In [59]:
# Import IMDB movie data
title_basics = pd.read_csv('../data/title.basics.tsv',delimiter='\t',encoding='utf-8-sig',low_memory=False)

# Import ratings
title_ratings = pd.read_csv('../data/title.ratings.tsv',delimiter='\t',encoding='utf-8-sig')

# Import crew
title_crew = pd.read_csv('../data/title.crew.tsv',delimiter='\t',encoding='utf-8-sig')

# Import name basics
name_basics = pd.read_csv('../data/name.basics.tsv',delimiter='\t',encoding='utf-8-sig')

# Import box office data from BoxOfficeMojo
box_office = pd.read_csv('../data/boxoffice.csv')

# Import Oscar data from Wikipedia: https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films
oscars = pd.read_csv('../data/oscars_cleaned.csv')

In [60]:
# Filter non-movies, adult movies
title_basics_filtered = title_basics[title_basics['titleType']=='movie']
title_basics_filtered = title_basics_filtered[title_basics_filtered['isAdult']==0]

In [61]:
# Split out genres
genres_split = title_basics_filtered["genres"].str.split(",", n=2, expand=True)

## REFACTOR, LOSING MOVIES, CHECK JOIN/MERGE
# Concatonate with main movies
joined = pd.concat([title_basics_filtered, genres_split], axis=1, sort=False)

In [86]:
joined.count()

Type             78552
Title            78552
isAdult          78552
Year             78552
Runtime (min)    78552
Genre (main)     78552
Genre (sub 1)    78552
Genre (sub 2)    78552
Rating (avg.)    55953
Votes            55953
dtype: int64

In [62]:
# Rename and drop some columns
joined = joined.rename(columns = {'tconst':'IMDB ID', 'titleType': 'Type', 'primaryTitle': 'Title', 'originalTitle': 'Title (original)', 'startYear': 'Year', 'runtimeMinutes': 'Runtime (min)', 0:'Genre (main)',1:'Genre (sub 1)', 2:'Genre (sub 2)'})
joined = joined.drop(columns=['endYear', 'genres', 'Title (original)'])

# Drop empty rows
joined = joined.dropna(axis='rows')

In [63]:
# Merge basic dataframe and rating dataframe
joined = joined.set_index('IMDB ID').join(title_ratings.set_index('tconst'))
joined = joined.rename(columns = {'averageRating': 'Rating (avg.)', 'numVotes': 'Votes'})

In [72]:
# Merge
box_office_oscars = box_office.join(oscars.set_index('Film'), on='title')
merged = joined.join(box_office_oscars.set_index('title'), on='Title', rsuffix='_right')

# Drop columns
merged = merged.drop(['Type', 'isAdult', 'Year_right', 'Genre (sub 1)', 'Genre (sub 2)', 'year', 'rank'], axis=1)

In [73]:
merged = merged.rename(columns = {'studio': 'Studio', 'lifetime_gross': 'Lifetime gross ($)'})

In [79]:
# FIX
#merged[['Votes']] = merged[['Votes']].apply(pd.to_numeric, downcast='integer')
merged.count()


# Budget

# Check for dupes
#merged = merged.dropna(axis='rows')

Title                 78671
Year                  78671
Runtime (min)         78671
Genre (main)          78671
Rating (avg.)         56051
Votes                 56051
Studio                 9024
Lifetime gross ($)     9032
Awards                  545
Nominations             545
dtype: int64

## STUFF NOT USING AT THE MOMENT

In [ ]:
# Calculate avg gross ($) per movie
box_plot['Avg. lifetime gross'] = box_plot['lifetime_gross']/box_plot['title']

# Reset index to get year in column
box_plot.reset_index()